<h1>Neural Network SMS Text Classifier ML </h1>

In [3]:
# import libraries

import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

2.16.1


In [4]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2024-04-12 20:05:54--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 2606:4700:20::ac43:4695, 2606:4700:20::681a:321, 2606:4700:20::681a:221, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|2606:4700:20::ac43:4695|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv      100%[===================>] 349,84K  --.-KB/s    in 0,02s   

2024-04-12 20:05:54 (21,1 MB/s) - ‘train-data.tsv’ saved [358233/358233]

--2024-04-12 20:05:54--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 2606:4700:20::681a:221, 2606:4700:20::681a:321, 2606:4700:20::ac43:4695, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|2606:4700:20::681a:221|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab

In [47]:
# Make dataframe of dataset to see data

dataset_train = pd.read_csv(train_file_path, sep='\t')

# Rename the first column to 'label' and second column to 'text' and reverse the order
list_columns = dataset_train.columns.tolist()
list_columns[1] = 'text'
list_columns[0] = 'label'
dataset_train.columns = list_columns

dataset_train = dataset_train.iloc[:, ::-1]
dataset_train.head(10)

,text,label
0,you can never do nothing,ham
1,"now u sound like manky scouse boy steve,like! ...",ham
2,mum say we wan to go then go... then she can s...,ham
3,never y lei... i v lazy... got wat? dat day ü ...,ham
4,in xam hall boy asked girl tell me the startin...,ham
5,genius what's up. how your brother. pls send h...,ham
6,they finally came to fix the ceiling.,ham
7,urgent! call 09066350750 from your landline. y...,spam
8,now that you have started dont stop. just pray...,ham
9,hello darlin ive finished college now so txt m...,ham


In [48]:
dataset_test = pd.read_csv(test_file_path, sep='\t')

# Rename the first column to 'label' and second column to 'text' and reverse the order
list_columns = dataset_test.columns.tolist()
list_columns[1] = 'text'
list_columns[0] = 'label'
dataset_test.columns = list_columns

dataset_test = dataset_test.iloc[:, ::-1]
dataset_test.head(10)

,text,label
0,"not much, just some textin'. how bout you?",ham
1,i probably won't eat at all today. i think i'm...,ham
2,don‘t give a flying monkeys wot they think and...,ham
3,who are you seeing?,ham
4,your opinion about me? 1. over 2. jada 3. kusr...,ham
5,yesterday its with me only . now am going home.,ham
6,yes. it's all innocent fun. o:-),ham
7,a boy was late 2 home. his father: power of fr...,ham
8,is ur changes 2 da report big? cos i've alread...,ham
9,sorry about earlier. putting out fires.are you...,ham


In [39]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):



  return (prediction)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

NameError: name 'prediction' is not defined

In [40]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


NameError: name 'prediction' is not defined